In [1]:
!pip3 install scikit-learn

In [2]:
import sklearn.linear_model
import numpy as np
import pandas as pd
import sklearn.linear_model

In [7]:
file_gdp="gdp-bli/gdp_per_capita_2014-2025.csv"
file_happy="gdp-bli/BLI_25112020150937807.csv"

df_gdp=pd.read_csv(file_gdp,thousands=',',delimiter='\t',encoding='latin1',na_values='n/a')
df_happy=pd.read_csv(file_happy,thousands=',')

In [8]:
rows=5
df_gdp.head(rows)

,Country,Subject Descriptor,Units,Scale,Country/Series-specific Notes,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,Estimates Start After
0,Afghanistan,"Gross domestic product per capita, current prices",U.S. dollars,Units,"See notes for: Gross domestic product, curren...",629.921,599.349,519.219,532.22,505.909,507.29,499.441,505.827,523.013,540.522,555.893,571.606,2019.0
1,Albania,"Gross domestic product per capita, current prices",U.S. dollars,Units,"See notes for: Gross domestic product, curren...",4584.920,3953.610,4124.410,4542.76,5284.440,5323.19,4898.280,5160.770,5647.780,6217.290,6623.230,7054.600,2018.0
2,Algeria,"Gross domestic product per capita, current prices",U.S. dollars,Units,"See notes for: Gross domestic product, curren...",5466.330,4153.320,3918.940,4012.13,4080.910,3898.01,3331.080,3449.420,3512.060,3541.220,3581.020,3641.010,2019.0
3,Angola,"Gross domestic product per capita, current prices",U.S. dollars,Units,"See notes for: Gross domestic product, curren...",5625.740,4354.920,3676.830,4303.69,3620.590,2967.96,2021.310,2129.750,2198.090,2317.600,2437.410,2542.210,2017.0
4,Antigua and Barbuda,"Gross domestic product per capita, current prices",U.S. dollars,Units,"See notes for: Gross domestic product, curren...",13951.990,14696.890,15556.010,15655.23,16860.960,17195.46,14158.570,14747.840,16368.890,17756.520,18558.790,19365.520,2011.0


In [9]:
def prepare_country_stats(data_happy,data_gdp):
    # Process happiness data
    # select rows matching TOT in columns INEQUALITY
    data_happy = data_happy[data_happy["INEQUALITY"]=="TOT"]
    # Create new df using 'Country' column as index and 'Indicator' as columns
    # df is populated using values from column 'Values'
    data_happy = data_happy.pivot(index="Country", columns="Indicator",values="Value")

    # Process GDP data
    # rename column and set a new index
    data_gdp = data_gdp.rename(columns={"2021":"GDP per capita (2020)"})
    data_gdp = data_gdp.set_index("Country")
    
    # Merge both datasets by country
    merged_data = pd.merge(left=data_happy, right=data_gdp,left_index=True, right_index=True)
    merged_data.sort_values(by="GDP per capita (2020)",inplace=True)
    
    # Create two sub datasets for testing and validation
    # We select two specific columns out of the merged dataset
    remove_indices = [0, 1, 6, 8, 33, 34, 35, 36, 37, 38, 39]
    keep_indices = list(set(range(merged_data.shape[0])) - set(remove_indices))
    
    test_data     = merged_data[["GDP per capita (2020)",'Life satisfaction']].iloc[keep_indices]
    validate_data = merged_data[["GDP per capita (2020)",'Life satisfaction']].iloc[remove_indices]
    
    return test_data, validate_data

In [10]:
country_stats_test, country_stats_validate = prepare_country_stats(df_happy, df_gdp)
x = np.c_[country_stats_test["GDP per capita (2020)"]]
y = np.c_[country_stats_test["Life satisfaction"]]